# KMP 算法笔记 

* KMP 算法名字由来: 三个学者名字的首字母拼成  
    * Knuth，Morris和Pratt，所以取了三位学者名字的首字母。所以叫做KMP



* KMP 算法解决的问题:
    * 字符串匹配问题

# KMP 算法核心 -- 前缀表 -- next数组就是一个前缀表（prefix table）。
* 前缀表的作用:
    * 记录下标i之前（包括i）的字符串中，有多大长度的相同前缀后缀。

* 前缀 的概念:
    * 字符串中, [包含第一个字符, 不包含最后一个字符 的子串] 都是 前缀

* 后缀 的概念:
    * 字符串中, [不含第一个字符, 一定包含最后一个字符 的子串] 都是 后缀

* 前缀表记录的是 '最长相等前后缀的长度' 构成的[序列]:
    * 前缀表记录的是 '最长相等前后缀'
    * [举例] 对于 字符串 "aabaaf" 来说:
        * 子串 "a": 没有前缀也没有后缀, 最长相等前后缀长度是 0
        * 子串 "aa": 前缀是[a], 后缀也是[a], 前缀后缀相等, 长度为1
        * 子串 "aab": 前缀是 [a, aa], 后缀是 [b, ab], 前缀后缀里没有相等的, 所以最长相等前后缀长度是 0
        * 子串 "aaba": 前缀是[a, aa, aab], 后缀是[a, ba, aba], 相等的只有 [a], 所以最长相等前后缀长度是 1
        * 子串 "aabaa": 前缀是 [a, aa, aab, aaba], 后缀是[a, aa, baa, abaa], 相等的有 [a, aa], 所以最长相等前后缀长度是 2
        * 子串 "aabaaf": 前缀是[a, aa, aab, aaba, aabaa], 后缀是[f, af, aaf, baaf, abaaf], 没有相等的, 最长相等前后缀长度是 0 
        * 此时得到 一个序列 [0,1,0,1,2,0], 每一个数字的 index 对应着子串的长度

## 继续上面的列子来说明 KMP算法 
* 文本串为: "aabaabaafa", 模式串为: "aabaaf", KMP算法匹配过程如下图
![jupyter](https://code-thinking.cdn.bcebos.com/gifs/KMP%E7%B2%BE%E8%AE%B21.gif)



* 算法中, 遇到f匹配失败时, 不再从头开始判断, 而是从[相同前缀]的下一个字母开始遍历, 这样可以节省时间.
    * 操作的原理:
        * 前缀表中记录了每一个字符 i (包含i) 之前的所有 [最长相等前后缀]的长度, 也就是说, 当遍历到例子中的f, 发现不匹配时, f 的前一个字符 a 在前缀表中记录的长度为2, 这说明 aabaa 这段子串的最后两个元素与最开始的两个元素相等, 那么我们就不需要再遍历 开头的aa了 (说明文本串中, f前面的两个aa已经遍历过), 此时, 前缀表中记录的长度刚好就是 下一次需要遍历的元素的下标 (因为index从0开始)

![jupyter](https://code-thinking.cdn.bcebos.com/gifs/KMP%E7%B2%BE%E8%AE%B22.gif)

# 前缀表与 next 数组

* 这并不涉及到 KMP 的原理，而是具体实现，next数组既可以就是前缀表，也可以是前缀表 **统一减一（右移一位，初始位置为-1）**。
    * 使用 next 数组时, 匹配的过程如下:

![jupyter](https://code-thinking.cdn.bcebos.com/gifs/KMP%E7%B2%BE%E8%AE%B24.gif)

# 时间复杂度分析:
* 假设 文本串的长度为n, 模板串的长度为m, 则:
    * 构造Next数组的时间复杂度度是 O(m)    -- 针对模板串构造 前缀表, 并统一减1
    * 匹配时, 是在不断遍历**文本串**, 然后 不断调整**模板串**中指针的位置, 因此 时间复杂度为O(n)
    * 一共是两个过程的顺序叠加, 并没有嵌套, 所以整体的时间复杂度为O(m)+O(n)

    * PS: 暴力解法的循环是有嵌套的, 所以时间复杂度是 O(m*n), 所以 KMP 算法更快.

# 构造Next数组  (构造前缀表)
* 思路 -- 处理以下三个步骤:
1. 初始化
2. 处理前后缀不相同的情况
3. 处理前后缀相同的情况


## 初始化:
* 定义两个指针 i 和 j
    * i 指向 **后缀**末尾位置
    * j 指向 **前缀**末尾位置                                        ==> 一般认为 j 刚好指停在(当前元素)前后缀最长子串的位置
        * j 初始化为 -1  (这里构造的是Next数组)
* 初始化Next数组, 数组长度与模板串长度一致
    * 说明: Next[xx] 表示 xx(包含xx) **之前**最长相等的前后缀长度       ==> 其实就是 j 的位置 
    * Next[0] = j 


## 处理前后缀不相同的情况
* 显然, 由于 i 指向 后缀末尾开始, 而后缀不能包含首字符, 因此, 遍历时 i 应该从 1 开始.
    * 遍历模式串s时, 把s[i] 与 s[j+1] 进行比较  (因为j初始化为-1, 所以这里+1)
        * 比较的时候, 逻辑是 由前后缀的末尾开始, 向前后缀的头部方向遍历, 逐一比较. 
* 当 s[i] 与 s[j+1] 不同时, 就要向 对应的前/后缀 头部方向移动   ==> 向前退回
    * 怎么回退呢？
        * next[j]就是记录着j（包括j）之前的子串的相同前后缀的长度。
        * 那么 s[i] 与 s[j+1] 不相同，就要找 j+1前一个元素在next数组里的值（就是next[j]）。
        * 因此, 退回时, j = next[j]

## 处理前后缀相同的情况
* 如果 s[i] 与 s[j + 1] 相同，那么就同时向后移动i 和j 说明找到了相同的前后缀，
* 同时还要将 j（前缀的长度）赋给next[i], 因为 next[i] 要记录相同前后缀的长度。


* 动画示意图:

![jupyter](https://code-thinking.cdn.bcebos.com/gifs/KMP%E7%B2%BE%E8%AE%B23.gif)


### 所以, 整体的 C++ 代码为:
```
void getNext(int* next, const string& s){
    int j = -1;
    next[0] = j;
    for(int i = 1; i < s.size(); i++) {         // 注意i从1开始
        while (j >= 0 && s[i] != s[j + 1]) {    // 前后缀不相同了
            j = next[j];                        // 向前回退
        }
        if (s[i] == s[j + 1]) {                 // 找到相同的前后缀
            j++;
        }
        next[i] = j;                            // 将j（前缀的长度）赋给next[i]
    }
}
```

# (已知前缀表) 用next数组做匹配
* 在文本串s里 找是否出现过模式串t。

* 定义两个下标 j 指向模式串起始位置，i 指向文本串起始位置。
    * 那么 j 初始值依然为 -1，为什么呢？ 
        * 依然因为next数组里记录的起始位置为-1。
    * 遍历时, i 从 0 开始 (i指向s里的第一个字符位置)

* 接下来, 判断 s[i] 与 t[j+1]    ==> 因为 j 初始值为-1
    * 如果 s[i] 与 t[j+1] 不相同, j 就从 next 数组里寻找下一个匹配的位置
        * j = next[j]   
    * 如果 s[i] 与 t[j+1] 相同, 则 i 和 j 都向后移动

* 判断 t 在 s 中出现过的条件:
    * t 顺利遍历到最后一个元素, 即 j = len(s)-1

* 返回匹配子串的第一个位置:
    * return  ( i - (len(t) -1) )
    * i 是文本串当前遍历到的位置, 减去 模板串的长度即可.



### 整体C++代码:
```
int j = -1;                                 // 因为next数组里记录的起始位置为-1
for (int i = 0; i < s.size(); i++) {        // 注意i就从0开始
    while(j >= 0 && s[i] != t[j + 1]) {     // 不匹配
        j = next[j];                        // j 寻找之前匹配的位置
    }
    if (s[i] == t[j + 1]) {                 // 匹配，j和i同时向后移动
        j++;                                // i的增加在for循环里
    }
    if (j == (t.size() - 1) ) {             // 文本串s里出现了模式串t
        return (i - t.size() + 1);
    }
}
```

# 前缀表减一 (next数组) 的完整C++ 实现

```
class Solution {
public:
    void getNext(int* next, const string& s) {
        int j = -1;
        next[0] = j;
        for(int i = 1; i < s.size(); i++) {             // 注意i从1开始
            while (j >= 0 && s[i] != s[j + 1]) {        // 前后缀不相同了
                j = next[j];                            // 向前回退
            }
            if (s[i] == s[j + 1]) {                     // 找到相同的前后缀
                j++;
            }
            next[i] = j;                                // 将j（前缀的长度）赋给next[i]
        }
    }
    int strStr(string haystack, string needle) {
        if (needle.size() == 0) {
            return 0;
        }
        int next[needle.size()];
        getNext(next, needle);
        int j = -1;                                             // 因为next数组里记录的起始位置为-1
        for (int i = 0; i < haystack.size(); i++) {             // 注意i就从0开始
            while(j >= 0 && haystack[i] != needle[j + 1]) {     // 不匹配
                j = next[j];                                    // j 寻找之前匹配的位置
            }
            if (haystack[i] == needle[j + 1]) {                 // 匹配，j和i同时向后移动
                j++;                                            // i的增加在for循环里
            }
            if (j == (needle.size() - 1) ) {                    // 文本串s里出现了模式串t
                return (i - needle.size() + 1);
            }
        }
        return -1;
    }
};
```

# 前缀表不减一 (原生前缀表)的完整 C++ 实现:

* 主要就是 j=next[x] 这一步最为关键！


## 前缀表的构建部分:

```
    void getNext(int* next, const string& s) {
        int j = 0;
        next[0] = 0;
        for(int i = 1; i < s.size(); i++) {
            while (j > 0 && s[i] != s[j]) {         // j要保证大于0，因为下面有取j-1作为数组下标的操作
                j = next[j - 1];                    // 注意这里，是要找前一位的对应的回退位置了
            }
            if (s[i] == s[j]) {
                j++;
            }
            next[i] = j;
        }
    }

```


## 完整代码:
```
class Solution {
public:
    void getNext(int* next, const string& s) {
        int j = 0;
        next[0] = 0;
        for(int i = 1; i < s.size(); i++) {             
            while (j > 0 && s[i] != s[j]) {             // j要保证大于0，因为下面有取j-1作为数组下标的操作
                j = next[j - 1];                        // 注意这里，是要找前一位的对应的回退位置了
            }
            if (s[i] == s[j]) {
                j++;
            }
            next[i] = j;
        }
    }
    int strStr(string haystack, string needle) {
        if (needle.size() == 0) {
            return 0;
        }
        int next[needle.size()];
        getNext(next, needle);
        int j = 0;
        for (int i = 0; i < haystack.size(); i++) { 
            while(j > 0 && haystack[i] != needle[j]) {  // j要保证大于0，因为下面有取j-1作为数组下标的操作
                j = next[j - 1];                        // 注意这里，是要找前一位的对应的回退位置了
            }
            if (haystack[i] == needle[j]) {
                j++;
            }
            if (j == needle.size() ) {
                return (i - needle.size() + 1);
            }
        }
        return -1;
    }
};
```

# python 版本代码一 (使用next数组实现 -- 前缀表统一减1)

In [7]:
# 方法一: 使用for循环遍历
class Solution:
    def strStr(self, haystack: str, needle: str) -> int:
        a = len(needle)
        b = len(haystack)
        if a == 0:
            return 0
        next = self.getnext(a,needle)       # 构建 next 数组
        p=-1                                # p 遍历 模板串
        for j in range(b):                  # j 遍历 文本串
            while p >= 0 and needle[p+1] != haystack[j]:        # 文本串与模板串当前遍历的字符不相同
                p = next[p]                                 # 模板串指针p 跳到 前一个元 素对应的最长相等前后缀长度位置
            if needle[p+1] == haystack[j]:                      # 文本串与模板串当前遍历的字符相同, 两个指针都 继续遍历下一个字符
                p += 1
            if p == a-1:                                        # 模板串遍历到末尾, 结束遍历, 返回 文本串中匹配的第一个位置
                return j-a+1
        return -1

    def getnext(self,a,needle):
        next = ['' for i in range(a)]           # 创建数组空间
        k = -1                                  # k 指向 前缀的末尾位置
        next[0] = k                             # 初始化 next 数组, 第一个元素毕竟没有最长相等前后缀, 初始化为-1
        for i in range(1, len(needle)):         # i 指向 后缀的末尾位置
            while (k > -1 and needle[k+1] != needle[i]):    # 前后缀 不相等时
                k = next[k]                                 # 前缀 后退 到前一个字符对应的最长相等前后缀位置
            if needle[k+1] == needle[i]:                    # 前后缀相等, 前缀后缀都继续 减少元素, 进行下一次遍历
                k += 1
            next[i] = k                                     # 得到 当前遍历的字符(包含i位置) 对应的 最长相等前后缀 长度
        return next

In [ ]:
# 方法二: 用while循环来写
class Solution:
    def strStr(self, haystack: str, needle: str) -> int:
        a = len(needle)
        b = len(haystack)
        if a == 0:
            return 0
        i = j = 0
        next = self.getnext(a, needle)              # 构建 next 数组
        while(i < b and j < a):
            if j == -1 or needle[j] == haystack[i]: # 文本串与模板串当前遍历的字符相同, 两个指针都 继续遍历下一个字符
                i += 1
                j += 1
            else:                                   # 文本串与模板串当前遍历的字符不相同
                j = next[j]                         # 模板串指针p 跳到 前一个元 素对应的最长相等前后缀长度位置
        if j == a:                                  # 模板串遍历到末尾, 结束遍历, 返回 文本串中匹配的第一个位置
            return i-j  
        else:
            return -1

    def getnext(self, a, needle):
        next = ['' for i in range(a)]
        j, k = 0, -1
        next[0] = k
        while(j < a-1):
            if k == -1 or needle[k] == needle[j]:
                k += 1
                j += 1
                next[j] = k
            else:
                k = next[k]
        return next

# python 版本代码二 (标准前缀表)

In [2]:
# 前缀表（不减一）Python实现
class Solution:
    def strStr(self, haystack: str, needle: str) -> int:
        if len(needle) == 0:
            return 0
        next = self.getNext(needle)
        j = 0
        for i in range(len(haystack)):
            while j >= 1 and haystack[i] != needle[j]:
                j = next[j-1]
            if haystack[i] == needle[j]:
                j += 1
            if j == len(needle):
                return i - len(needle) + 1
        return -1
    
    def getNext(self, needle):
        next = [0] * len(needle)
        j = 0
        next[0] = j
        for i in range(1, len(needle)):
            while j >= 1 and needle[i] != needle[j]:
                j = next[j-1]
            if needle[i] == needle[j]:
                j += 1
            next[i] = j
        return next

In [10]:
_test = Solution()
haystack = "sadbutsad"; needle = "sad"
_test.getnext(len(needle),needle) 

[-1, -1, -1]